In [10]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
import xlwings as xw
import tempfile
import os
import re
import quantstats as qs

# ------------- 1. Excel Integration & Configuration ----------------

excel_file = 'Richspread.xlsx'
sheet_name = 'Port_Optimization'

# --- Cell locations for outputs ---
simulation_chart_cell = 'A10'
optimization_table_cell = 'A40'
quantstats_chart_cell = 'Q4'
quantstats_metrics_cell = 'Q40'

print(f"Connecting to '{excel_file}'...")
book = xw.Book(excel_file)
sht = book.sheets[sheet_name]

# --- Clear previous outputs from the sheet ---
print("Clearing previous results from Excel sheet...")
try:
    for pic in sht.pictures:
        if pic.name in ['PortfolioOptimization', 'PortfolioSnapshot']:
            pic.delete()
except Exception:
    pass # No pictures to delete
sht.range(optimization_table_cell).expand().clear_contents()
sht.range(quantstats_chart_cell).expand().clear_contents()
sht.range(quantstats_metrics_cell).expand().clear_contents()


# --- Read inputs from Excel ---
print("Reading inputs from Excel...")
tickers = sht.range('B7:AA7').value
start = sht.range('B2').value
end = sht.range('B3').value
max_vol = sht.range('B4').value
min_ror_constraint = sht.range('B5').value
simulation_numbers = 10000

# --- Validate and prepare tickers ---
flat_tickers = [t for t in tickers if isinstance(t, str) and re.match(r'^[A-Za-z0-9\.\-\=]+$', t)]
if not flat_tickers:
    raise ValueError("No valid tickers found in the Excel sheet. Please check the range B7:AA7.")

print("Tickers:", flat_tickers)
print("Start Date:", start)
print("End Date:", end)
print("Max Volatility Constraint:", max_vol)
print("Min RoR Constraint:", min_ror_constraint)

# --- Download historical data ---
print("Downloading historical stock data...")
df = yf.download(flat_tickers, start=start, end=end, actions=False)['Close']
df = df.sort_index()
log_returns = np.log(df / df.shift(1)).dropna()


# ------------- 2. Portfolio Optimization Functions ----------------

def portfolio_return(weights, log_returns):
    return np.sum(log_returns.mean() * weights) * 252

def portfolio_volatility(weights, log_returns):
    return np.sqrt(np.dot(weights.T, np.dot(log_returns.cov() * 252, weights)))

def minimize_negative_sharpe(weights, log_returns):
    ror = portfolio_return(weights, log_returns)
    vol = portfolio_volatility(weights, log_returns)
    return -(ror / vol) if vol != 0 else -np.inf

def optimize_portfolio(log_returns, objective_function, constraints=None, bounds=None):
    num_assets = len(log_returns.columns)
    if bounds is None:
        bounds = tuple((0, 1) for _ in range(num_assets))
    if constraints is None:
        constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
    
    initial_guess = num_assets * [1. / num_assets,]
    opt_results = minimize(objective_function, initial_guess, args=(log_returns,), method='SLSQP', bounds=bounds, constraints=constraints)
    return opt_results

# ------------- 3. Monte Carlo Simulation & Optimization ----------------

print("\nRunning Monte Carlo simulation...")
weights_sim = np.random.random((simulation_numbers, len(flat_tickers)))
weights_sim /= np.sum(weights_sim, axis=1)[:, np.newaxis]

ror_sim = np.array([portfolio_return(w, log_returns) for w in weights_sim])
vol_sim = np.array([portfolio_volatility(w, log_returns) for w in weights_sim])
sharpe_sim = ror_sim / vol_sim

# --- Run Mathematical Optimizations ---
print("Running mathematical optimizations...")
# Base constraints and bounds
cons_base = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
bnds = tuple((0, 1) for _ in range(len(flat_tickers)))

# Max Sharpe Ratio
opt_sharpe = optimize_portfolio(log_returns, minimize_negative_sharpe, constraints=cons_base, bounds=bnds)
# Min Volatility
opt_vol = optimize_portfolio(log_returns, portfolio_volatility, constraints=cons_base, bounds=bnds)
# Max Return
opt_ror = optimize_portfolio(log_returns, lambda w, r: -portfolio_return(w, r), constraints=cons_base, bounds=bnds)
# Constrained: Max RoR for given Volatility
cons_max_ror = cons_base + [{'type': 'ineq', 'fun': lambda x: max_vol - portfolio_volatility(x, log_returns)}]
opt_max_ror_constrained = optimize_portfolio(log_returns, lambda w, r: -portfolio_return(w, r), constraints=cons_max_ror, bounds=bnds)
# Constrained: Min Vol for given RoR
cons_min_vol = cons_base + [{'type': 'ineq', 'fun': lambda x: portfolio_return(x, log_returns) - min_ror_constraint}]
opt_min_vol_constrained = optimize_portfolio(log_returns, portfolio_volatility, constraints=cons_min_vol, bounds=bnds)


# ------------- 4. Export Optimization Results to Excel ----------------

# --- Create and export simulation scatter plot ---
print("Exporting simulation plot to Excel...")
with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmpfile:
    chart_path = tmpfile.name
    plt.figure(figsize=(8, 4))
    plt.scatter(vol_sim, ror_sim, c=sharpe_sim, cmap='viridis', alpha=0.6, label='Simulated Portfolios')
    plt.colorbar(label='Sharpe Ratio')
    # Plot optimal points
    plt.scatter(portfolio_volatility(opt_sharpe.x, log_returns), portfolio_return(opt_sharpe.x, log_returns), marker='*', color='red', s=200, label='Max Sharpe Ratio')
    plt.scatter(portfolio_volatility(opt_vol.x, log_returns), portfolio_return(opt_vol.x, log_returns), marker='*', color='black', s=200, label='Min Volatility')
    plt.scatter(portfolio_volatility(opt_ror.x, log_returns), portfolio_return(opt_ror.x, log_returns), marker='*', color='blue', s=200, label='Max Return')
    plt.title('Portfolio Optimization Simulation')
    plt.xlabel('Volatility (Annualized)')
    plt.ylabel('Return (Annualized)')
    plt.legend()
    plt.tight_layout()
    plt.savefig(chart_path, bbox_inches='tight', facecolor='white')
    plt.close()
    sht.pictures.add(chart_path, name="PortfolioOptimization", update=True, 
                     left=sht.range(simulation_chart_cell).left, 
                     top=sht.range(simulation_chart_cell).top, 
                     width=500, height=300)
    try:
        os.remove(chart_path)
    except Exception as e:
        print(f"Warning: Could not delete temp chart file: {e}")

# --- Create and export optimization summary table ---
print("Exporting optimization summary table to Excel...")
def get_opt_dict(title, opt_result, log_returns, tickers):
    weights = opt_result.x
    ror = portfolio_return(weights, log_returns)
    vol = portfolio_volatility(weights, log_returns)
    sharpe = ror / vol if vol != 0 else float('nan')
    weights_dict = {ticker: round(weight, 4) for ticker, weight in zip(tickers, weights)}
    return {"Portfolio": title, **weights_dict, "RoR": round(ror, 4), "Volatility": round(vol, 4), "Sharpe Ratio": round(sharpe, 4)}

opt_rows = [
    get_opt_dict("Optimal (Max Sharpe Ratio)", opt_sharpe, log_returns, flat_tickers),
    get_opt_dict("Optimal (Max RoR)", opt_ror, log_returns, flat_tickers),
    get_opt_dict("Optimal (Min Volatility)", opt_vol, log_returns, flat_tickers),
    get_opt_dict(f"Optimal (Max RoR, Vol ≤ {max_vol:.2f})", opt_max_ror_constrained, log_returns, flat_tickers),
    get_opt_dict(f"Optimal (Min Vol, RoR ≥ {min_ror_constraint:.2f})", opt_min_vol_constrained, log_returns, flat_tickers),
]

opt_df = pd.DataFrame(opt_rows)
# Reorder columns to have metrics at the end
cols_to_move = ['RoR', 'Volatility', 'Sharpe Ratio']
new_cols_order = [c for c in opt_df.columns if c not in cols_to_move] + cols_to_move
opt_df = opt_df[new_cols_order]

sht.range(optimization_table_cell).options(index=False).value = opt_df


# ------------- 5. QuantStats Analysis for Max Sharpe Portfolio ----------------

print("\nRunning QuantStats analysis on the Max Sharpe Ratio portfolio...")
# Use the weights from the best portfolio (Max Sharpe)
max_sharpe_weights = opt_sharpe.x
daily_returns = df.pct_change().dropna()
portfolio_returns_qs = daily_returns.dot(max_sharpe_weights)

# --- Generate and export QuantStats plot ---
print("Exporting QuantStats performance plot...")
with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmpfile:
    qs_plot_path = tmpfile.name
    fig = qs.plots.snapshot(portfolio_returns_qs, title="Max Sharpe Portfolio Performance", show=False)
    fig.savefig(qs_plot_path, bbox_inches='tight')
    plt.close(fig)
    sht.pictures.add(qs_plot_path, name='PortfolioSnapshot', update=True, left=sht.range(quantstats_chart_cell).left, top=sht.range(quantstats_chart_cell).top, height=450, width=900)
    try:
        os.remove(qs_plot_path)
    except Exception as e:
        print(f"Warning: Could not delete temp qs chart file: {e}")

# --- Generate and export QuantStats metrics ---
print("Exporting QuantStats metrics table...")
metrics_df = qs.reports.metrics(portfolio_returns_qs, benchmark="SPY", mode="full", display=False, prepare_returns=False)
sht.range(quantstats_metrics_cell).value = "Performance Metrics (Max Sharpe Portfolio vs SPY)"
sht.range(quantstats_metrics_cell).offset(row_offset=1).options(pd.DataFrame, index=True).value = metrics_df

print("\n--- Process Complete ---")
print("Successfully exported all results to the 'Port_Optimization' sheet.")

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
import xlwings as xw
import tempfile
import os
import re
import quantstats as qs

# ------------- 1. Excel Integration & Configuration ----------------

excel_file = 'Richspread.xlsx'
sheet_name = 'Port_Optimization'

# --- Cell locations for outputs ---
simulation_chart_cell = 'A10'
optimization_table_cell = 'A40'
quantstats_chart_cell = 'Q4'
quantstats_metrics_cell = 'Q40'

print(f"Connecting to '{excel_file}'...")
book = xw.Book(excel_file)
sht = book.sheets[sheet_name]

# --- Clear previous outputs from the sheet ---
print("Clearing previous results from Excel sheet...")
try:
    for pic in sht.pictures:
        if pic.name in ['PortfolioOptimization', 'PortfolioSnapshot']:
            pic.delete()
except Exception:
    pass # No pictures to delete
sht.range(optimization_table_cell).expand().clear_contents()
sht.range(quantstats_chart_cell).expand().clear_contents()
sht.range(quantstats_metrics_cell).expand().clear_contents()


# --- Read inputs from Excel ---
print("Reading inputs from Excel...")
tickers = sht.range('B7:AA7').value
start = sht.range('B2').value
end = sht.range('B3').value
max_vol = sht.range('B4').value
min_ror_constraint = sht.range('B5').value
simulation_numbers = 10000

# --- Validate and prepare tickers ---
flat_tickers = [t for t in tickers if isinstance(t, str) and re.match(r'^[A-Za-z0-9\.\-\=]+$', t)]
if not flat_tickers:
    raise ValueError("No valid tickers found in the Excel sheet. Please check the range B7:AA7.")

print("Tickers:", flat_tickers)
print("Start Date:", start)
print("End Date:", end)
print("Max Volatility Constraint:", max_vol)
print("Min RoR Constraint:", min_ror_constraint)

# --- Download historical data ---
print("Downloading historical stock data...")
df = yf.download(flat_tickers, start=start, end=end, actions=False)['Close']
df = df.sort_index()
log_returns = np.log(df / df.shift(1)).dropna()


# ------------- 2. Portfolio Optimization Functions ----------------

def portfolio_return(weights, log_returns):
    return np.sum(log_returns.mean() * weights) * 252

def portfolio_volatility(weights, log_returns):
    return np.sqrt(np.dot(weights.T, np.dot(log_returns.cov() * 252, weights)))

def minimize_negative_sharpe(weights, log_returns):
    ror = portfolio_return(weights, log_returns)
    vol = portfolio_volatility(weights, log_returns)
    return -(ror / vol) if vol != 0 else -np.inf

def optimize_portfolio(log_returns, objective_function, constraints=None, bounds=None):
    num_assets = len(log_returns.columns)
    if bounds is None:
        bounds = tuple((0, 1) for _ in range(num_assets))
    if constraints is None:
        constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
    
    initial_guess = num_assets * [1. / num_assets,]
    opt_results = minimize(objective_function, initial_guess, args=(log_returns,), method='SLSQP', bounds=bounds, constraints=constraints)
    return opt_results

# ------------- 3. Monte Carlo Simulation & Optimization ----------------

print("\nRunning Monte Carlo simulation...")
weights_sim = np.random.random((simulation_numbers, len(flat_tickers)))
weights_sim /= np.sum(weights_sim, axis=1)[:, np.newaxis]

ror_sim = np.array([portfolio_return(w, log_returns) for w in weights_sim])
vol_sim = np.array([portfolio_volatility(w, log_returns) for w in weights_sim])
sharpe_sim = ror_sim / vol_sim

# --- Run Mathematical Optimizations ---
print("Running mathematical optimizations...")
# Base constraints and bounds
cons_base = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
bnds = tuple((0, 1) for _ in range(len(flat_tickers)))

# Max Sharpe Ratio
opt_sharpe = optimize_portfolio(log_returns, minimize_negative_sharpe, constraints=cons_base, bounds=bnds)
# Min Volatility
opt_vol = optimize_portfolio(log_returns, portfolio_volatility, constraints=cons_base, bounds=bnds)
# Max Return
opt_ror = optimize_portfolio(log_returns, lambda w, r: -portfolio_return(w, r), constraints=cons_base, bounds=bnds)
# Constrained: Max RoR for given Volatility
cons_max_ror = cons_base + [{'type': 'ineq', 'fun': lambda x: max_vol - portfolio_volatility(x, log_returns)}]
opt_max_ror_constrained = optimize_portfolio(log_returns, lambda w, r: -portfolio_return(w, r), constraints=cons_max_ror, bounds=bnds)
# Constrained: Min Vol for given RoR
cons_min_vol = cons_base + [{'type': 'ineq', 'fun': lambda x: portfolio_return(x, log_returns) - min_ror_constraint}]
opt_min_vol_constrained = optimize_portfolio(log_returns, portfolio_volatility, constraints=cons_min_vol, bounds=bnds)


# ------------- 4. Export Optimization Results to Excel ----------------

# --- Create and export simulation scatter plot ---
print("Exporting simulation plot to Excel...")
with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmpfile:
    chart_path = tmpfile.name
    plt.figure(figsize=(10, 6))
    plt.scatter(vol_sim, ror_sim, c=sharpe_sim, cmap='viridis', alpha=0.6, label='Simulated Portfolios')
    plt.colorbar(label='Sharpe Ratio')
    # Plot optimal points
    plt.scatter(portfolio_volatility(opt_sharpe.x, log_returns), portfolio_return(opt_sharpe.x, log_returns), marker='*', color='red', s=200, label='Max Sharpe Ratio')
    plt.scatter(portfolio_volatility(opt_vol.x, log_returns), portfolio_return(opt_vol.x, log_returns), marker='*', color='black', s=200, label='Min Volatility')
    plt.scatter(portfolio_volatility(opt_ror.x, log_returns), portfolio_return(opt_ror.x, log_returns), marker='*', color='blue', s=200, label='Max Return')
    plt.title('Portfolio Optimization Simulation')
    plt.xlabel('Volatility (Annualized)')
    plt.ylabel('Return (Annualized)')
    plt.legend()
    plt.tight_layout()
    plt.savefig(chart_path, bbox_inches='tight', facecolor='white')
    plt.close()
    sht.pictures.add(chart_path, name="PortfolioOptimization", update=True, left=sht.range(simulation_chart_cell).left, top=sht.range(simulation_chart_cell).top, width=500, height=300)
    try:
        os.remove(chart_path)
    except Exception as e:
        print(f"Warning: Could not delete temp chart file: {e}")

# --- Create and export optimization summary table ---
print("Exporting optimization summary table to Excel...")
def get_opt_dict(title, opt_result, log_returns, tickers):
    weights = opt_result.x
    ror = portfolio_return(weights, log_returns)
    vol = portfolio_volatility(weights, log_returns)
    sharpe = ror / vol if vol != 0 else float('nan')
    weights_dict = {ticker: round(weight, 4) for ticker, weight in zip(tickers, weights)}
    return {"Portfolio": title, **weights_dict, "RoR": round(ror, 4), "Volatility": round(vol, 4), "Sharpe Ratio": round(sharpe, 4)}

opt_rows = [
    get_opt_dict("Optimal (Max Sharpe Ratio)", opt_sharpe, log_returns, flat_tickers),
    get_opt_dict("Optimal (Max RoR)", opt_ror, log_returns, flat_tickers),
    get_opt_dict("Optimal (Min Volatility)", opt_vol, log_returns, flat_tickers),
    get_opt_dict(f"Optimal (Max RoR, Vol ≤ {max_vol:.2f})", opt_max_ror_constrained, log_returns, flat_tickers),
    get_opt_dict(f"Optimal (Min Vol, RoR ≥ {min_ror_constraint:.2f})", opt_min_vol_constrained, log_returns, flat_tickers),
]

opt_df = pd.DataFrame(opt_rows)
# Reorder columns to have metrics at the end
cols_to_move = ['RoR', 'Volatility', 'Sharpe Ratio']
new_cols_order = [c for c in opt_df.columns if c not in cols_to_move] + cols_to_move
opt_df = opt_df[new_cols_order]

sht.range(optimization_table_cell).options(index=False).value = opt_df


# ------------- 5. QuantStats Analysis for Max Sharpe Portfolio ----------------

print("\nRunning QuantStats analysis on the Max Sharpe Ratio portfolio...")
# Use the weights from the best portfolio (Max Sharpe)
max_sharpe_weights = opt_sharpe.x
daily_returns = df.pct_change().dropna()
portfolio_returns_qs = daily_returns.dot(max_sharpe_weights)

# --- Generate and export QuantStats plot ---
print("Exporting QuantStats performance plot...")
with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmpfile:
    qs_plot_path = tmpfile.name
    fig = qs.plots.snapshot(portfolio_returns_qs, title="Max Sharpe Portfolio Performance", show=False)
    fig.savefig(qs_plot_path, bbox_inches='tight')
    plt.close(fig)
    sht.pictures.add(qs_plot_path, name='PortfolioSnapshot', update=True, left=sht.range(quantstats_chart_cell).left, top=sht.range(quantstats_chart_cell).top, height=450, width=900)
    try:
        os.remove(qs_plot_path)
    except Exception as e:
        print(f"Warning: Could not delete temp qs chart file: {e}")

# --- Generate and export QuantStats metrics ---
print("Exporting QuantStats metrics table...")
metrics_df = qs.reports.metrics(portfolio_returns_qs, benchmark="SPY", mode="full", display=False, prepare_returns=False)
sht.range(quantstats_metrics_cell).value = "Performance Metrics (Max Sharpe Portfolio vs SPY)"
sht.range(quantstats_metrics_cell).offset(row_offset=1).options(pd.DataFrame, index=True).value = metrics_df

print("\n--- Process Complete ---")
print("Successfully exported all results to the 'Port_Optimization' sheet.")

Connecting to 'Richspread.xlsx'...
Clearing previous results from Excel sheet...
Reading inputs from Excel...
Tickers: ['AAPL', 'GME', 'MSFT', 'TSLA', 'JPM', 'SNAP']
Start Date: 2022-12-31 00:00:00
End Date: 2025-07-14 00:00:00
Max Volatility Constraint: 0.5
Min RoR Constraint: 0.15


[*********************100%***********************]  6 of 6 completed



Running Monte Carlo simulation...
Running mathematical optimizations...
Exporting simulation plot to Excel...
Exporting optimization summary table to Excel...

Running QuantStats analysis on the Max Sharpe Ratio portfolio...
Exporting QuantStats performance plot...
Exporting QuantStats metrics table...

--- Process Complete ---
Successfully exported all results to the 'Port_Optimization' sheet.
Connecting to 'Richspread.xlsx'...
Clearing previous results from Excel sheet...


[*********************100%***********************]  6 of 6 completed

Reading inputs from Excel...
Tickers: ['AAPL', 'GME', 'MSFT', 'TSLA', 'JPM', 'SNAP']
Start Date: 2022-12-31 00:00:00
End Date: 2025-07-14 00:00:00
Max Volatility Constraint: 0.5
Min RoR Constraint: 0.15

Running Monte Carlo simulation...


Running mathematical optimizations...
Exporting simulation plot to Excel...
Exporting optimization summary table to Excel...

Running QuantStats analysis on the Max Sharpe Ratio portfolio...
Exporting QuantStats performance plot...
Exporting QuantStats metrics table...

--- Process Complete ---
Successfully exported all results to the 'Port_Optimization' sheet.
